# <b>MODIS Water Validation Notebook - Compare models</b>

Purpose: Used to perform validation of C61 MOD44W products from different models. Compares those products to the previous version, C6 MOD44W.

*Note: We are following an incremental development lifecycle. This notebook is the first rendition which fit most of the requirements. Expect incremental releases which continue towards the goal of fully meeting requirements and increasing capabilities of the user.*

Installation requirements:

```bash
pip install localtileserver
```

TODO:
- ipysheet for user to input comments
- load layers from toolbar
- move everything inside a class to avoid user input

Some references:

- https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
- https://github.com/giswqs/geodemo/blob/master/geodemo/common.py

Version: 0.0.1
Date: 12/09/2022

*For DSG internal use*

### <b> WARNING </b>

Do not run all cells at once, doing so will shut down the local tile servers before you, the user, can interact.

Uncomment if localtileserver is not installed

In [1]:
# !pip install localtileserver

In [2]:
import os
import re
import json
import joblib
import tempfile
import ipysheet
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
import warnings
import tempfile

from osgeo import gdal
from pprint import pprint

from glob import glob
from ipysheet import from_dataframe
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl, AwesomeIcon
from ipyleaflet import LegendControl, FullScreenControl, MarkerCluster, Popup

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

import localtileserver
from localtileserver import get_leaflet_tile_layer, TileClient

## Tile and year selection

Choose which tile (see MODIS grid) and which year. Reference the grid image. 

The `h` followed by two numerical digits represent the <b>horizontal</b> tile ID. Use the column space to determine this ID. 

The `v` followed by two numerical digits represent the <b>vertical</b> tile ID. Use the row space to determine this ID. 

For example, the tile that is 9 columns to the right and 5 rows down is `h09v05`.

Example:
```python
TILE = 'h09v05'
```

![MODIS Grid Overlay](../imgs/modis_overlay.png)

In [3]:
# TILE = 'h09v05'
# TILE = 'h11v02'
TILE = 'h11v10'
# TILE = 'h12v09'
# TILE = 'h16v02'
# TILE = 'h17v02'
# TILE = 'h18v03'
# TILE = 'h21v10'
# TILE = 'h22v01'
# TILE = 'h27v03'
# TILE = 'h28v08'
# TILE = 'h30v11'

In [4]:
YEAR = 2019

Shouldn't need to change anything under this 

In [5]:
MOD44W_C6_BASEPATH = '/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs/'
MOD44W_C61_VERSION = '001'
C6_FILE_TYPE = '.tif'

C61_RF_FILE_TYPE = '.tif'
TMP_FILE_TYPE = '.tif'

HDF_PRESTR = 'HDF4_EOS:EOS_GRID'
HDF_POSSTR = 'MOD44W_250m_GRID'

SEVEN_CLASS = 'seven_class'
WATER_MASK = 'water_mask'
WATER_MASK_QA = 'water_mask_QA'

if YEAR > 2019:
    warnings.warn('Using 2019 C6 MOD44W')
    MOD44_C6_YEAR = 2019
else:
    MOD44_C6_YEAR = YEAR

tiles_basemap: str = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
water_rf_v1_cmap: list = ['#E3B878', '#2d7d86']
water_rf_v2_cmap: list = ['#194d33', '#8ed1fc']
water_c6_cmap: list = ['#194d33', '#7bdc93']
water_rfa_qa_cmap: list = ['#ee82ee', '#FCB900', '#FF6900', '#800080']
water_qa_cmap: list = ['#79d2a6', '#ff6900', '#e4efe9']
perm_water_cmap: list = ['#b8174e', '#00e202']

CACHE_DIR = '.cache'
os.makedirs(CACHE_DIR, exist_ok=True)

mod44w_c6_path = os.path.join(MOD44W_C6_BASEPATH, str(MOD44_C6_YEAR), f'MOD44W_{TILE}_{MOD44_C6_YEAR}_v5.tif')
if not os.path.exists(mod44w_c6_path):
    raise FileNotFoundError(f'Could not find the MOD44W C6 file: {mod44w_c6_path}')
    
crs = 'PROJCS["Sinusoidal",GEOGCS["Sphere",DATUM["Sphere",SPHEROID["Sphere",6371000,0]],PRIMEM["Greenwich",0],' + \
    'UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"]' + \
    ',PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0]' + \
',UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
mod44w_cs_ds = gdal.Open(mod44w_c6_path)
transform = mod44w_cs_ds.GetGeoTransform()
mod44w_c6_ds = None

In [6]:
def parse_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 0, 0, -1)
    qa_array_parsed = xr.where(qa_array == 4, 1, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 3, qa_array_parsed)
    return qa_array_parsed

def parse_fix_qa(qa_array):
    qa_array_parsed = xr.where(qa_array == 2, 1, 0)
    qa_array_parsed = xr.where(qa_array == 4, 2, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 6, 3, qa_array_parsed)
    qa_array_parsed = xr.where(qa_array == 9, 4, qa_array_parsed)
    return qa_array_parsed

def open_and_write_temp(data_array, transform, projection, 
                        year, tile, name = None, files_to_rm = None) -> str:
    tmpdir = tempfile.gettempdir()
    name_to_use = data_array.name if not name else name
    tempfile_name = f'MOD44W.A{year}001.{tile}.061.{name_to_use}.tif'
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    if os.path.exists(tempfile_fp):
        os.remove(tempfile_fp)
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(tempfile_fp, 4800, 4800, 
                          1, gdal.GDT_Float32, 
                          options=['COMPRESS=LZW'])
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(projection)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(data_array.data[0, :, :])
    outBand.SetNoDataValue(250)
    outDs.FlushCache()
    outDs = None
    outBand = None
    driver = None
    return tempfile_fp

def get_location(cache_dir: str, tile: str, def_location: list) -> list:
    cache_fp = os.path.join(cache_dir, f'{tile}.marker.location.sv')
    if os.path.exists(cache_fp):
        location = joblib.load(cache_fp)
    else:
        location = def_location
    return location

def cache_location(tile: str, location: list) -> None:
    cache_fp = os.path.join(CACHE_DIR, f'{tile}.marker.location.sv')
    output = joblib.dump(location, cache_fp)
    return None

def initialize_marker(tile: str, location: list, cache_dir: str) -> Marker:
    name = 'Location Marker'
    title = name
    location = get_location(cache_dir, tile, location)
    marker = Marker(name=name, title=name, location=location)
    return marker

def initialize_message(location: list) -> widgets.HTML:
    ll_message = widgets.HTML()
    ll_message.value = str(location)
    return ll_message

## Adding a new product (ANP)

Add a new basepath which points to a dir that contains the MODIS output tifs

```python
MOD44W_RFA_V5_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Amanda_Comparison_04042023/1_2_7_NDVI_v3'
```

In [7]:
MOD44W_RFA_V1_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Amanda_Comparison_04042023/RFA_NEW'
MOD44W_RFA_V2_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Amanda_Comparison_04042023/1_2_7_RFA'
MOD44W_RFA_V3_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Amanda_Comparison_04042023/1_2_7_RFA_v000'
MOD44W_RFA_V4_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/Amanda_Comparison_04042023/1_2_7_NDVI_v201'

## !! ANP !!

Copy a code block and change to fit what version

e.g.

```python
mod44w_rfa_v5_regex = os.path.join(MOD44W_RFA_V5_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProduct*{C61_RF_FILE_TYPE}')
mod44w_rfa_v5_qa_regex = os.path.join(MOD44W_RFA_V5_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProductQA*{C61_RF_FILE_TYPE}')
```

In [8]:
mod44w_rfa_v1_regex = os.path.join(MOD44W_RFA_V1_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProduct*{C61_RF_FILE_TYPE}')
mod44w_rfa_v1_qa_regex = os.path.join(MOD44W_RFA_V1_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProductQA*{C61_RF_FILE_TYPE}')

mod44w_rfa_v2_regex = os.path.join(MOD44W_RFA_V2_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProduct*{C61_RF_FILE_TYPE}')
mod44w_rfa_v2_qa_regex = os.path.join(MOD44W_RFA_V2_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProductQA*{C61_RF_FILE_TYPE}')

mod44w_rfa_v3_regex = os.path.join(MOD44W_RFA_V3_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProduct*{C61_RF_FILE_TYPE}')
mod44w_rfa_v3_qa_regex = os.path.join(MOD44W_RFA_V3_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProductQA*{C61_RF_FILE_TYPE}')

mod44w_rfa_v4_regex = os.path.join(MOD44W_RFA_V4_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProduct*{C61_RF_FILE_TYPE}')
mod44w_rfa_v4_qa_regex = os.path.join(MOD44W_RFA_V4_BASEPATH,
                     f'MOD44W.A{YEAR}.{TILE}.RandomForest.AnnualWaterProductQA*{C61_RF_FILE_TYPE}')

## !! ANP !!

Copy and add a code block for your new version

```python
mod44w_rfa_v5_path = sorted(glob(mod44w_rfa_v5_regex))[0]
mod44w_rfa_v5_qa_path = sorted(glob(mod44w_rfa_v5_qa_regex))[0]
```

Do this with all these code blocks

In [9]:
mod44w_rfa_v1_path = sorted(glob(mod44w_rfa_v1_regex))[0]
mod44w_rfa_v1_qa_path = sorted(glob(mod44w_rfa_v1_qa_regex))[0]

mod44w_rfa_v2_path = sorted(glob(mod44w_rfa_v2_regex))[0]
mod44w_rfa_v2_qa_path = sorted(glob(mod44w_rfa_v2_qa_regex))[0]

mod44w_rfa_v3_path = sorted(glob(mod44w_rfa_v3_regex))[0]
mod44w_rfa_v3_qa_path = sorted(glob(mod44w_rfa_v3_qa_regex))[0]

mod44w_rfa_v4_path = sorted(glob(mod44w_rfa_v4_regex))[0]
mod44w_rfa_v4_qa_path = sorted(glob(mod44w_rfa_v4_qa_regex))[0]

## !! ANP !!

In [10]:
mod44w_rfa_v1_data_array = rxr.open_rasterio(mod44w_rfa_v1_path)
mod44w_rfa_v2_data_array = rxr.open_rasterio(mod44w_rfa_v2_path)
mod44w_rfa_v3_data_array = rxr.open_rasterio(mod44w_rfa_v3_path)
mod44w_rfa_v4_data_array = rxr.open_rasterio(mod44w_rfa_v4_path)

## !! ANP !!

In [11]:
mod44w_rfa_v1_qa_data_array = rxr.open_rasterio(mod44w_rfa_v1_qa_path)
mod44w_rfa_v1_perm_water_array = parse_fix_qa(mod44w_rfa_v1_qa_data_array)

mod44w_rfa_v2_qa_data_array = rxr.open_rasterio(mod44w_rfa_v2_qa_path)
mod44w_rfa_v2_perm_water_array = parse_fix_qa(mod44w_rfa_v2_qa_data_array)

mod44w_rfa_v3_qa_data_array = rxr.open_rasterio(mod44w_rfa_v3_qa_path)
mod44w_rfa_v3_perm_water_array = parse_fix_qa(mod44w_rfa_v3_qa_data_array)

mod44w_rfa_v4_qa_data_array = rxr.open_rasterio(mod44w_rfa_v4_qa_path)
mod44w_rfa_v4_perm_water_array = parse_fix_qa(mod44w_rfa_v4_qa_data_array)

## !! ANP !!

Copy and change the variable name but also remember to change the input data array name and the name argument

e.g.
```python
mod44w_rfa_v5_water_mask = open_and_write_temp(mod44w_rfa_v5_data_array, transform, crs,  YEAR, TILE, name='v5_mask', files_to_rm=temporary_files_to_delete)
mod44w_rfa_v5_perm_water_mask_qa = open_and_write_temp(mod44w_rfa_v5_perm_water_array, transform, crs, YEAR, TILE, name='v5_perm_qa_mask', files_to_rm=temporary_files_to_delete)
```

In [12]:
temporary_files_to_delete = []

mod44w_rfa_v1_water_mask = open_and_write_temp(mod44w_rfa_v1_data_array, transform, crs, YEAR, TILE, name='v1_mask', files_to_rm=temporary_files_to_delete)
mod44w_rfa_v1_perm_water_mask_qa = open_and_write_temp(mod44w_rfa_v1_perm_water_array, transform, crs,  YEAR, TILE, name='v1_perm_qa_mask', files_to_rm= temporary_files_to_delete)

mod44w_rfa_v2_water_mask = open_and_write_temp(mod44w_rfa_v2_data_array, transform, crs,  YEAR, TILE, name='v2_mask', files_to_rm=temporary_files_to_delete)
mod44w_rfa_v2_perm_water_mask_qa = open_and_write_temp(mod44w_rfa_v2_perm_water_array, transform, crs, YEAR, TILE, name='v2_perm_qa_mask', files_to_rm=temporary_files_to_delete)

mod44w_rfa_v3_water_mask = open_and_write_temp(mod44w_rfa_v3_data_array, transform, crs,  YEAR, TILE, name='v3_mask', files_to_rm=temporary_files_to_delete)
mod44w_rfa_v3_perm_water_mask_qa = open_and_write_temp(mod44w_rfa_v3_perm_water_array, transform, crs, YEAR, TILE, name='v3_perm_qa_mask', files_to_rm=temporary_files_to_delete)

mod44w_rfa_v4_water_mask = open_and_write_temp(mod44w_rfa_v4_data_array, transform, crs,  YEAR, TILE, name='v4_mask', files_to_rm=temporary_files_to_delete)
mod44w_rfa_v4_perm_water_mask_qa = open_and_write_temp(mod44w_rfa_v4_perm_water_array, transform, crs, YEAR, TILE, name='v4_perm_qa_mask', files_to_rm=temporary_files_to_delete)

## !! ANP !!

Copy the set for each of these and change version name

In [13]:
mod44w_c6_client = TileClient(mod44w_c6_path)
mod44w_rfa_v1_water_client = TileClient(mod44w_rfa_v1_water_mask)
mod44w_rfa_v1_perm_water_client = TileClient(mod44w_rfa_v1_perm_water_mask_qa)
mod44w_rfa_v2_water_client = TileClient(mod44w_rfa_v2_water_mask)
mod44w_rfa_v2_perm_water_client = TileClient(mod44w_rfa_v2_perm_water_mask_qa)
mod44w_rfa_v3_water_client = TileClient(mod44w_rfa_v3_water_mask)
mod44w_rfa_v3_perm_water_client = TileClient(mod44w_rfa_v3_perm_water_mask_qa)
mod44w_rfa_v4_water_client = TileClient(mod44w_rfa_v4_water_mask)
mod44w_rfa_v4_perm_water_client = TileClient(mod44w_rfa_v4_perm_water_mask_qa)

## !! ANP !!

Copy the set for each of these and change version name as well as add description for layer

In [14]:
mod44w_c6_water_mask_layer = get_leaflet_tile_layer(
    mod44w_c6_client, nodata=0, show=False, 
    vmin=0, vmax=1,
    cmap=water_c6_cmap, 
    name=f'MOD44W C6 Water Mask {YEAR} {TILE}',
    max_zoom=20)

mod44w_rfa_v1_water_mask_layer = get_leaflet_tile_layer(
    mod44w_rfa_v1_water_client, nodata=0, show=False,
    vmin=0, vmax=1,
    cmap=water_rf_v1_cmap, 
    name=f'MOD44W RFA (V1) {YEAR} {TILE}',
    max_zoom=20)

mod44w_rfa_v1_perm_water_layer = get_leaflet_tile_layer(
    mod44w_rfa_v1_perm_water_client, nodata=0, show=False,
    vmin=1, vmax=4,
    cmap=water_rfa_qa_cmap, 
    name=f'MOD44W RFA (V1) Permanent Water Mask {TILE}',
    max_zoom=20)

mod44w_rfa_v2_water_mask_layer = get_leaflet_tile_layer(
    mod44w_rfa_v2_water_client, nodata=0, show=False,
    vmin=0, vmax=1,
    cmap=water_rf_v2_cmap, 
    name=f'MOD44W RFA (1-2-7) {YEAR} {TILE}',
    max_zoom=20)

mod44w_rfa_v2_perm_water_layer = get_leaflet_tile_layer(
    mod44w_rfa_v2_perm_water_client, nodata=0, show=False,
    vmin=1, vmax=4,
    cmap=water_rfa_qa_cmap, 
    name=f'MOD44W RFA (1-2-7) Permanent Water Mask {TILE}',
    max_zoom=20)

mod44w_rfa_v3_water_mask_layer = get_leaflet_tile_layer(
    mod44w_rfa_v3_water_client, nodata=0, show=False,
    vmin=0, vmax=1,
    cmap=water_rf_v2_cmap, 
    name=f'MOD44W RFA (1-2-7 v000) {YEAR} {TILE}',
    max_zoom=20)

mod44w_rfa_v3_perm_water_layer = get_leaflet_tile_layer(
    mod44w_rfa_v3_perm_water_client, nodata=0, show=False,
    vmin=1, vmax=4,
    cmap=water_rfa_qa_cmap, 
    name=f'MOD44W RFA (1-2-7 v000) Permanent Water Mask {TILE}',
    max_zoom=20)

mod44w_rfa_v4_water_mask_layer = get_leaflet_tile_layer(
    mod44w_rfa_v4_water_client, nodata=0, show=False,
    vmin=0, vmax=1,
    cmap=water_rf_v2_cmap, 
    name=f'MOD44W RFA (1-2-7-NDVI v201) {YEAR} {TILE}',
    max_zoom=20)

mod44w_rfa_v4_perm_water_layer = get_leaflet_tile_layer(
    mod44w_rfa_v4_perm_water_client, nodata=0, show=False,
    vmin=1, vmax=4,
    cmap=water_rfa_qa_cmap, 
    name=f'MOD44W RFA (1-2-7-NDVI V201) Permanent Water Mask {TILE}',
    max_zoom=20)

## !! ANP !!

Add a new legend dict for your new product, make sure to update the legend_dict with that new sub-dict

```python
rfa_v5_water_mask_legend_dict = {'RFA (NEW BAND v201)- Water': '#8ed1fc'}

...


legend_dict.update(rfa_v5_water_mask_legend_dict)

```

In [15]:
legend_dict = {}

c6_water_mask_legend_dict = {'C6- Water': '#7bdc93'}
rfa_v1_water_mask_legend_dict = {'RFA V1- Water': '#2d7d86'}
rfa_v2_water_mask_legend_dict = {'RFA (1-2-7)- Water': '#8ed1fc'}
rfa_v3_water_mask_legend_dict = {'RFA (1-2-7 v000)- Water': '#8ed1fc'}
rfa_v4_water_mask_legend_dict = {'RFA (1-2-7-NDVI v201)- Water': '#8ed1fc'}

qa_water_legend_dict = {'QA- Perm Water Flipped L->W': '#ee82ee', # FF6900
                                 'QA- Ocean Mask L->W': '#FCB900',
                                 'QA- Burn Scar W->L': '#FF6900',
                                 'QA- DEM Slope Change W->L': '#800080'}

rfa_v2_perm_water_legend_dict = {'RFA (1-2-7)- Perm Water Flipped': '#00e202'}
legend_dict.update(c6_water_mask_legend_dict)

legend_dict.update(rfa_v1_water_mask_legend_dict)
legend_dict.update(rfa_v2_water_mask_legend_dict)
legend_dict.update(rfa_v3_water_mask_legend_dict)
legend_dict.update(rfa_v4_water_mask_legend_dict)
legend_dict.update(qa_water_legend_dict)

legend = LegendControl(legend_dict)

## !! ANP !!

Add a new layer with the correct version number

```python
m.add_layer(mod44w_rfa_v5_perm_water_layer)

```

In [16]:
m = Map(
    center=mod44w_c6_client.center(),
    zoom=mod44w_c6_client.default_zoom,
    basemap=basemaps.Esri.WorldImagery,
    scroll_wheel_zoom=True,
    keyboard=True,
    layout=widgets.Layout(height='600px')
)
marker_location = mod44w_c6_client.center()
marker = initialize_marker(tile=TILE, location=marker_location, cache_dir=CACHE_DIR)
latlon_message = initialize_message(marker.location)

def handle_click(**kwargs):
    latlon_message.value = str(marker.location)
    marker.popup = latlon_message
    cache_location(tile=TILE, location=marker.location)

m.add_layer(marker)
marker.on_click(handle_click)
m.add_layer(mod44w_c6_water_mask_layer)
m.add_layer(mod44w_rfa_v1_water_mask_layer)
m.add_layer(mod44w_rfa_v2_water_mask_layer)
m.add_layer(mod44w_rfa_v3_water_mask_layer)
m.add_layer(mod44w_rfa_v4_water_mask_layer)
m.add_layer(mod44w_rfa_v1_perm_water_layer)
m.add_layer(mod44w_rfa_v2_perm_water_layer)
m.add_layer(mod44w_rfa_v3_perm_water_layer)
m.add_layer(mod44w_rfa_v4_perm_water_layer)
m.add_control(legend)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

## MODIS Water Validation Map Visualization

<b>Usage Tips:</b>

- ![Layer Control](../imgs/layer_control.png)    Hover over to select and deselect which layers are visible

- ![Full Screen Control](../imgs/full_screen.png)    Click for full screen

- Use the scroll wheel on the mouse to zoom in and out, or use [+] and [-]

The legend shows all layers no matter what's visible but each element is prefixed with which layer it indicates. I.e.: 

- "Seven Class-": MOD44W C61 Seven Class

- "QA-": MOD44W C61 QA Mask

- "C6-": MOD44W C6 Water Mask

- "C61-": MOD44W C61 Water Mask

- "Difference-": MOD44W Difference

In [17]:
display(m)

Map(center=[-15.00001637063555, -67.70910047688926], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [19]:
userid = !whoami
notes_path = f'../notes/{TILE}-{userid[0]}-notes.csv'
if os.path.exists(notes_path):
    notes_df = pd.read_csv(notes_path)
    notes_df = notes_df.drop(columns=['Unnamed: 0'])
    sheet_notes = ipysheet.from_dataframe(notes_df)
else:
    tile = [' ' for _ in range(75)]
    year = [' ' for _ in range(75)]
    location = [' ' for _ in range(75)]
    note = [' ' for _ in range(75)]
    data = {'Tile': tile, 'Year': year, 'Location': location, 'Note': note}
    notes_df = pd.DataFrame(data=data)
    sheet_notes = ipysheet.from_dataframe(notes_df)
sheet_notes.column_width = [3,3,4,10]
sheet_notes.layout = widgets.Layout(width='100%',height='100%')
sheet_notes

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=74, row_start=0, squeeze_row=Fals…

## Save notes

Run this cell to save notes in the current working directory

In [19]:
sheet_notes_df = ipysheet.to_dataframe(sheet_notes)
sheet_notes_df.to_csv(notes_path)

### <b>DO NOT RUN THIS CELL UNTIL FINISHED WITH VALIDATION</b>
*Note: This will shut down the local tile servers*

*Ignore warnings as such:*
```
Server for key (default) not found.
```

In [18]:
for path_to_delete in temporary_files_to_delete:
    if os.path.exists(path_to_delete):
        os.remove(path_to_delete)
    temporary_files_to_delete.remove(path_to_delete)

mod44w_rfa_v1_water_client.shutdown(True)